# Binary Classification with CNN vs. ML Classifiers

This notebook demonstrates how to design, train, and evaluate a Convolutional Neural Network (CNN) to classify images as **with mask** or **without mask**. We also explore hyperparameter variations (learning rate, batch size, optimizer, and activation function) and finally compare the CNN's performance with traditional ML classifiers using handcrafted features.

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import to_categorical

from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

import tensorflow as tf

c:\Users\varni\miniconda3\envs\myenv\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.5 when it was built against 1.14.2, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


## 1. Data Loading and Preprocessing

We assume your dataset is organized into two folders: one for images **with mask** and another for images **without mask**. We load the images in color, resize them to 64x64, and normalize pixel values.

In [2]:
def load_images_from_folder(folder, label, image_size=(64, 64)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)  # Load in color
        if img is None:
            print(f"Warning: Unable to read {img_path}. Skipping...")
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        img = cv2.resize(img, image_size)
        images.append(img)
        labels.append(label)
    return images, labels

# Update these paths to point to your dataset folders
mask_folder = "./dataset/with_mask"
no_mask_folder = "./dataset/without_mask"

mask_images, mask_labels = load_images_from_folder(mask_folder, label=1)
no_mask_images, no_mask_labels = load_images_from_folder(no_mask_folder, label=0)

print(f"Loaded {len(mask_images)} images with mask and {len(no_mask_images)} images without mask.")

Loaded 2142 images with mask and 1930 images without mask.


In [3]:
# Combine the data
X = np.array(mask_images + no_mask_images)
y = np.array(mask_labels + no_mask_labels)

# Normalize pixel values
X = X.astype('float32') / 255.0

print('Total images:', X.shape[0])
print('Image shape:', X.shape[1:])

Total images: 4072
Image shape: (64, 64, 3)


## 2. Split Data into Training and Testing Sets

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Training set size:', X_train.shape[0])
print('Testing set size:', X_test.shape[0])

Training set size: 3257
Testing set size: 815


## 3. Building the CNN Model

We create a function `build_cnn_model` that accepts several hyperparameters:
- **learning_rate**: for the optimizer
- **optimizer_choice**: e.g., `'adam'` or `'sgd'`
- **batch_size**: used during training
- **activation**: activation function for the final classification layer (for binary classification, typically `sigmoid` is used)

For binary classification, we use a final Dense layer with 1 neuron and a `sigmoid` activation. You can experiment with other activations (though for binary tasks, `sigmoid` is most common).

In [5]:
def build_cnn_model(learning_rate=0.001, optimizer_choice='adam', final_activation='sigmoid', input_shape=(64, 64, 3)):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    
    # For binary classification, output one neuron
    model.add(Dense(1, activation=final_activation))
    
    if optimizer_choice.lower() == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_choice.lower() == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    else:
        raise ValueError("Unsupported optimizer choice")
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build a baseline model
baseline_model = build_cnn_model()
baseline_model.summary()

c:\Users\varni\miniconda3\envs\myenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 683,329 (2.61 MB)

 Trainable params: 683,329 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

## 4. Training the CNN Model with Hyperparameter Variations

Here we train the CNN using various hyperparameters. You can experiment by changing the learning rate, optimizer, batch size, or even the final activation (though for binary classification, `sigmoid` is typical). In this example, we run a few experiments and store the test accuracy for comparison.

In [ ]:
# Define hyperparameter experiments
experiments = [
    {'name': 'Baseline', 'learning_rate': 0.001, 'optimizer': 'adam', 'batch_size': 32, 'final_activation': 'sigmoid'},
    {'name': 'Low LR', 'learning_rate': 0.0001, 'optimizer': 'adam', 'batch_size': 32, 'final_activation': 'sigmoid'},
    {'name': 'High LR', 'learning_rate': 0.01, 'optimizer': 'adam', 'batch_size': 32, 'final_activation': 'sigmoid'},
    {'name': 'SGD Optimizer', 'learning_rate': 0.001, 'optimizer': 'sgd', 'batch_size': 32, 'final_activation': 'sigmoid'},
    {'name': 'Larger Batch', 'learning_rate': 0.001, 'optimizer': 'adam', 'batch_size': 64, 'final_activation': 'sigmoid'}
]

results = {}
num_epochs = 50  # For demonstration; increase epochs as needed

for exp in experiments:
    print(f"\nRunning experiment: {exp['name']}")
    model = build_cnn_model(learning_rate=exp['learning_rate'], optimizer_choice=exp['optimizer'], final_activation=exp['final_activation'])
    
    history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=exp['batch_size'], 
                        validation_data=(X_test, y_test), verbose=2)
    
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    results[exp['name']] = test_acc
    print(f"Test Accuracy for {exp['name']}: {test_acc:.4f}")

print("\nSummary of CNN Experiments:")
for name, acc in results.items():
    print(f"{name}: {acc:.4f}")


Running experiment: Baseline
Epoch 1/100
102/102 - 28s - 273ms/step - accuracy: 0.7731 - loss: 0.4525 - val_accuracy: 0.8908 - val_loss: 0.2805
Epoch 2/100
102/102 - 17s - 167ms/step - accuracy: 0.9002 - loss: 0.2696 - val_accuracy: 0.9264 - val_loss: 0.2170
Epoch 3/100
102/102 - 19s - 182ms/step - accuracy: 0.9239 - loss: 0.2099 - val_accuracy: 0.9387 - val_loss: 0.1731
Epoch 4/100
102/102 - 18s - 177ms/step - accuracy: 0.9355 - loss: 0.1834 - val_accuracy: 0.9337 - val_loss: 0.1756
Epoch 5/100
102/102 - 26s - 251ms/step - accuracy: 0.9429 - loss: 0.1650 - val_accuracy: 0.9362 - val_loss: 0.1579
Epoch 6/100
102/102 - 17s - 171ms/step - accuracy: 0.9512 - loss: 0.1402 - val_accuracy: 0.9607 - val_loss: 0.1197
Epoch 7/100
102/102 - 20s - 191ms/step - accuracy: 0.9613 - loss: 0.1158 - val_accuracy: 0.9227 - val_loss: 0.1940
Epoch 8/100
102/102 - 12s - 118ms/step - accuracy: 0.9598 - loss: 0.1103 - val_accuracy: 0.9632 - val_loss: 0.1187
Epoch 9/100
102/102 - 13s - 123ms/step - accuracy:

## 5. Comparing CNN Performance with ML Classifiers

In this section we extract handcrafted HOG features and train an SVM and a simple Neural Network (MLP) classifier. We then print classification reports for comparison.

Note: Ensure that the image size used for the CNN (64x64) is consistent with the one used for handcrafted feature extraction.

In [7]:
def load_images_grayscale(folder, label, image_size=(64, 64)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  
        if img is None:
            continue
        img = cv2.resize(img, image_size)
        images.append(img)
        labels.append(label)
    return images, labels

# Reload images in grayscale for handcrafted features
mask_images_gray, mask_labels_gray = load_images_grayscale(mask_folder, label=1)
no_mask_images_gray, no_mask_labels_gray = load_images_grayscale(no_mask_folder, label=0)

X_gray = np.array(mask_images_gray + no_mask_images_gray)
y_gray = np.array(mask_labels_gray + no_mask_labels_gray)

print('Grayscale images:', X_gray.shape)

Grayscale images: (4072, 64, 64)


In [8]:
# Extract HOG features
def extract_hog_features(images):
    hog_features = []
    for img in images:
        features = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
        hog_features.append(features)
    return np.array(hog_features)

X_hog = extract_hog_features(X_gray)

# Split for ML classifiers
X_train_hog, X_test_hog, y_train_hog, y_test_hog = train_test_split(X_hog, y_gray, test_size=0.2, random_state=42)

In [ ]:
# Train SVM classifier
svm_model = SVC(kernel='rbf',C=10)
svm_model.fit(X_train_hog, y_train_hog)
svm_pred = svm_model.predict(X_test_hog)
print("SVM Classifier Report:")
print(classification_report(y_test_hog, svm_pred))

# Train simple Neural Network classifier
nn_model = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500)
nn_model.fit(X_train_hog, y_train_hog)
nn_pred = nn_model.predict(X_test_hog)
print("Neural Network (MLP) Classifier Report:")
print(classification_report(y_test_hog, nn_pred))

SVM Classifier Report:
              precision    recall  f1-score   support

           0       0.86      0.89      0.87       362
           1       0.91      0.89      0.90       453

    accuracy                           0.89       815
   macro avg       0.88      0.89      0.88       815
weighted avg       0.89      0.89      0.89       815

Neural Network (MLP) Classifier Report:
              precision    recall  f1-score   support

           0       0.90      0.92      0.91       362
           1       0.93      0.92      0.92       453

    accuracy                           0.92       815
   macro avg       0.91      0.92      0.92       815
weighted avg       0.92      0.92      0.92       815



## Conclusion

In this notebook we:
- Loaded and preprocessed the dataset.
- Built and trained a CNN model with configurable hyperparameters.
- Ran experiments with different hyperparameter settings and recorded test accuracies.
- Compared the CNN's performance with ML classifiers using handcrafted HOG features.

Feel free to modify hyperparameters and architecture to further improve the performance!